Task 1: Grid and Obstacle SetupWe are using a 200x200 grid where most cells are walkable1. There are four specific square obstacles that block our path2. We allow movement in 8 directions: straight moves cost 1, and diagonal moves cost $\sqrt{2}$

In [12]:
import math
import heapq
import time

# Basic setup
grid_size = 200
start_node = (0, 0) # [cite: 4]
goal_node = (200, 200) # [cite: 5]

# Square obstacles defined in the prompt [cite: 11, 12, 13, 14]
blocked_zones = [
    (40, 40, 70, 70),
    (90, 120, 130, 160),
    (150, 50, 180, 90),
    (60, 150, 100, 190)
]

# Create the grid (0 is free, 1 is obstacle) [cite: 16]
my_map = [[0 for _ in range(grid_size + 1)] for _ in range(grid_size + 1)]
for x1, y1, x2, y2 in blocked_zones:
    for i in range(x1, x2 + 1):
        for j in range(y1, y2 + 1):
            my_map[i][j] = 1

def check_pos(x, y):
    # Make sure we are in bounds and not hitting a wall
    if 0 <= x <= grid_size and 0 <= y <= grid_size:
        return my_map[x][y] == 0
    return False

Task 2: Implementing Dijkstra's Algorithm
Dijkstra's will find the shortest path by exploring nodes based on their distance from the start.

In [13]:
def run_dijkstra(start, goal):
    count = 0
    todo_list = [(0.0, start)]
    dist_to_here = {start: 0.0}
    
    # 8-way movement costs [cite: 7, 8]
    steps = [
        (1, 0, 1), (-1, 0, 1), (0, 1, 1), (0, -1, 1),
        (1, 1, 1.414), (1, -1, 1.414), (-1, 1, 1.414), (-1, -1, 1.414)
    ]

    while todo_list:
        d, curr = heapq.heappop(todo_list)
        
        if d > dist_to_here.get(curr, 1e9):
            continue
            
        count += 1 # Tracking expanded nodes [cite: 20]
        if curr == goal:
            return d, count
            
        for dx, dy, cost in steps:
            nxt = (curr[0] + dx, curr[1] + dy)
            if check_pos(nxt[0], nxt[1]):
                new_d = d + cost
                if new_d < dist_to_here.get(nxt, 1e9):
                    dist_to_here[nxt] = new_d
                    heapq.heappush(todo_list, (new_d, nxt))
    return -1, count

# Run and report [cite: 19, 20]
t1 = time.time()
d_cost, d_expanded = run_dijkstra(start_node, goal_node)
t2 = time.time()

print(f"Dijkstra Cost: {d_cost}")
print(f"Nodes Expanded: {d_expanded}")
print(f"Time Taken: {t2-t1:.4f}s")

Dijkstra Cost: 300.9659999999986
Nodes Expanded: 34928
Time Taken: 0.1160s


Task 3: A* Algorithm
For A*, I designed a heuristic based on Octile distance. It calculates the distance assuming we move diagonally as much as possible before moving straight to the goal

In [14]:
def my_heuristic(p1, p2):
    # My custom heuristic for 8-direction movement 
    dx = abs(p1[0] - p2[0])
    dy = abs(p1[1] - p2[1])
    return (dx + dy) + (1.414 - 2) * min(dx, dy)

def run_astar(start, goal):
    nodes_count = 0
    # (f_score, g_score, position)
    open_nodes = [(my_heuristic(start, goal), 0.0, start)]
    g_costs = {start: 0.0}
    
    steps = [
        (1, 0, 1), (-1, 0, 1), (0, 1, 1), (0, -1, 1),
        (1, 1, 1.414), (1, -1, 1.414), (-1, 1, 1.414), (-1, -1, 1.414)
    ]

    while open_nodes:
        f, g, curr = heapq.heappop(open_nodes)
        
        if g > g_costs.get(curr, 1e9):
            continue
            
        nodes_count += 1
        if curr == goal:
            return g, nodes_count
            
        for dx, dy, cost in steps:
            nxt = (curr[0] + dx, curr[1] + dy)
            if check_pos(nxt[0], nxt[1]):
                new_g = g + cost
                if new_g < g_costs.get(nxt, 1e9):
                    g_costs[nxt] = new_g
                    new_f = new_g + my_heuristic(nxt, goal)
                    heapq.heappush(open_nodes, (new_f, new_g, nxt))
    return -1, nodes_count

# Run and report [cite: 25, 26]
t3 = time.time()
a_cost, a_expanded = run_astar(start_node, goal_node)
t4 = time.time()

print(f"A* Cost: {a_cost}")
print(f"Nodes Expanded: {a_expanded}")
print(f"Time Taken: {t4-t3:.4f}s")

A* Cost: 300.9659999999986
Nodes Expanded: 4469
Time Taken: 0.0279s


Comparison and Conclusion 


Path Length: Both found the same shortest path cost (~300.965 or slightly more depending on obstacle clearance).


Expanded Nodes: A* expanded way fewer nodes because the heuristic pointed it in the right direction.


Efficiency: A* is much faster for this kind of problem.


Heuristic Comment: I believe my heuristic does not overestimate the cost because Octile distance is the exact distance if there were no obstacles. Since obstacles only make the path longer, the estimate is always less than or equal to the real cost